<h1 style='color: #c52b10; font-size: 34px; font-weight: bold;'>Sistema de Recomendação de Filmes

</h1>


# <font color="#10c586" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #10c586;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b> Desenvolver um sistema de recomendação de filmes com base em características como gênero, palavras-chave, popularidade e classificação média, etc. </b></i>     
</p>

---

# <font color="#10c586" style="font-size: 30px;">Roteiro</font>
<hr style="border: 2px solid #10c586;">

<h1 style='color: black; font-size:22px; font-weight: bold;'> 1. Pré-processamento de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'>2. Criação da matriz de características

</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
3. Modelagem de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
4. Comunicação de Resultados
</h1>

---


# <font color="#10c586" style="font-size: 30px;">1. Pré-processamento de dados</font>
<hr style="border: 2px solid #10c586;">

In [291]:
# IMPORTANDO BIBLIOTECAS

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [292]:
# IMPORTANDO BASE DE DADOS

df = pd.read_csv('tmdb_5000_movies.csv')

df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [293]:
# RENOMEANDO AS COLUNAS

traducao = {
    'budget': 'Orcamento',
    'genres': 'Gêneros',
    'homepage': 'Página inicial',
    'id': 'ID',
    'keywords': 'Palavras-chave',
    'original_language': 'Idioma original',
    'original_title': 'Título original',
    'overview': 'Visão geral',
    'popularity': 'Popularidade',
    'production_companies': 'Empresas de produção',
    'production_countries': 'Países de produção',
    'release_date': 'Data de lançamento',
    'revenue': 'Receita',
    'runtime': 'Duração',
    'spoken_languages': 'Idiomas falados',
    'status': 'Status',
    'tagline': 'Slogan',
    'title': 'Título',
    'vote_average': 'Media_de_votos',
    'vote_count': 'Contagem_de_votos'
}

df = df.rename(columns=traducao)
df.head(2)

,Orcamento,Gêneros,Página inicial,ID,Palavras-chave,Idioma original,Título original,Visão geral,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Status,Slogan,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


---
## <font color=#c5106e style="font-size: 30px;">Transformar coluna de Data de lançamento em datetime</font>

---

In [294]:
# DANDO UMA OLHADA NA COLUNA DE LANÇAMENTOS

display(type(df['Data de lançamento'][0]))

#Converter representação de data em formato de string para um objeto de data no Python
 
from datetime import datetime

df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento']) 

display(type(df['Data de lançamento'][0]))

str

pandas._libs.tslibs.timestamps.Timestamp

```python
# Transforma a lista de listas em uma lista simples
lista_simples = [item for sublist in lista_generos_filmes for item in sublist]

# Obtém os valores únicos da lista
valores_unicos = set(lista_simples)

df['Gêneros'] = lista_simples[:4803]
```

---
## <font color=#c5106e style="font-size: 30px;">Removendo Dados inconsistentes</font>

---

In [295]:
df.describe()

,Orcamento,ID,Popularidade,Receita,Duração,Media_de_votos,Contagem_de_votos
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


---
## <font color=#c5106e style="font-size: 30px;">Não vamos querer nenhum filme com:</font>
---

*  <font color=#c5106e style="font-size: 28px;">Orçamento Zero</font>

---

* <font color=#c5106e style="font-size: 28px;">Receita Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Duração Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Média de votos Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Contagem de votos Zero</font>

---

In [296]:
df = df.query('Orcamento > 0 and Receita > 0 and Duração > 0 and Media_de_votos > 0 and Contagem_de_votos > 0')

---
## <font color=#c5106e style="font-size: 30px;">Mais de 75% dos filmes tem mais de 100 votos. Vamos filtrar</font>
---

In [297]:
df = df.query('Contagem_de_votos >= 100') # para assim só termos filmes com estatísticas relevantes de nota 

In [298]:
df.drop(['Página inicial', 'ID', 'Visão geral', 'Slogan', 'Status', 'Palavras-chave'], axis=1, inplace=True)
df

,Orcamento,Gêneros,Idioma original,Título original,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",en,Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",en,The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,4000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",en,The Signal,27.662696,"[{""name"": ""Automatik Entertainment"", ""id"": 281...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2014-03-15,600896,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Signal,5.8,631
4773,27000,"[{""id"": 35, ""name"": ""Comedy""}]",en,Clerks,19.748658,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1994-09-13,3151130,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Clerks,7.4,755
4788,12000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...",en,Pink Flamingos,4.553644,"[{""name"": ""Dreamland Productions"", ""id"": 407}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1972-03-12,6000000,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pink Flamingos,6.2,110
4796,7000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",en,Primer,23.307949,"[{""name"": ""Thinkfilm"", ""id"": 446}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2004-10-08,424760,77.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Primer,6.9,658


In [299]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2715 entries, 0 to 4798
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Orcamento             2715 non-null   int64         
 1   Gêneros               2715 non-null   object        
 2   Idioma original       2715 non-null   object        
 3   Título original       2715 non-null   object        
 4   Popularidade          2715 non-null   float64       
 5   Empresas de produção  2715 non-null   object        
 6   Países de produção    2715 non-null   object        
 7   Data de lançamento    2715 non-null   datetime64[ns]
 8   Receita               2715 non-null   int64         
 9   Duração               2715 non-null   float64       
 10  Idiomas falados       2715 non-null   object        
 11  Título                2715 non-null   object        
 12  Media_de_votos        2715 non-null   float64       
 13  Contagem_de_votos 

---
## <font color=#c5106e style="font-size: 30px;">Conformar a coluna de Gêneros, Empresas de produção, Países de produção, Idiomas falados para a forma que queremos</font>

---

In [300]:
def colunas_para_lista(df, colunas):
    lista_resultante = []
    for coluna in colunas:
        lista_coluna = df[coluna]
        lista_coluna = list(lista_coluna)
        lista_resultante.append(lista_coluna)
    return lista_resultante

# Lista de colunas que desejamos transformar em listas
colunas_transformar = ['Gêneros', 'Empresas de produção', 'Países de produção', 'Idiomas falados']

# Chamada da função para obter as listas
lista_generos, lista_empresas, lista_paises, lista_idiomas = colunas_para_lista(df, colunas_transformar)

import json


# Função para extrair os nomes dos gêneros de uma string em formato JSON
def extrair_dados(dados_str):
    dados_json = json.loads(dados_str)
    return [dados['name'] for dados in dados_json]

i = 0

for dados in colunas_para_lista(df, colunas_transformar):
    
    lista_dados_filmes = [extrair_dados(dados_str) for dados_str in dados]
    df[colunas_transformar[i]] = lista_dados_filmes
    i=i+1
df

,Orcamento,Gêneros,Idioma original,Título original,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",en,Avatar,150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",en,Pirates of the Caribbean: At World's End,139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",en,Spectre,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]",en,The Dark Knight Rises,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",en,John Carter,43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,4000000,"[Thriller, Science Fiction]",en,The Signal,27.662696,"[Automatik Entertainment, Low Spark Films, Sig...",[United States of America],2014-03-15,600896,95.0,[English],The Signal,5.8,631
4773,27000,[Comedy],en,Clerks,19.748658,"[Miramax Films, View Askew Productions]",[United States of America],1994-09-13,3151130,92.0,[English],Clerks,7.4,755
4788,12000,"[Horror, Comedy, Crime]",en,Pink Flamingos,4.553644,[Dreamland Productions],[United States of America],1972-03-12,6000000,93.0,[English],Pink Flamingos,6.2,110
4796,7000,"[Science Fiction, Drama, Thriller]",en,Primer,23.307949,[Thinkfilm],[United States of America],2004-10-08,424760,77.0,[English],Primer,6.9,658


---
## <font color=#c5106e style="font-size: 30px;">Idioma original: substituir siglas</font>

---

In [301]:
df['Idioma original'].unique()

dicionario_idiomas = {
    'en': 'Inglês',
    'ja': 'Japonês',
    'fr': 'Francês',
    'zh': 'Chinês',
    'ko': 'Coreano',
    'te': 'Telugu',
    'cn': 'Chinês',
    'es': 'Espanhol',
    'it': 'Italiano',
    'nl': 'Holandês',
    'ru': 'Russo',
    'de': 'Alemão',
    'th': 'Tailandês',
    'xx': 'Desconhecido',
    'da': 'Dinamarquês',
    'nb': 'Norueguês',
    'pt': 'Português',
    'pl': 'Polonês',
    'he': 'Hebraico',
    'hi': 'Hindi',
    'id': 'Indonésio',
    'ro': 'Romeno',
    'no': 'Norueguês',
    'fa': 'Persa'
}

df['Idioma original'] = df['Idioma original'].map(dicionario_idiomas)
df.columns

Index(['Orcamento', 'Gêneros', 'Idioma original', 'Título original',
       'Popularidade', 'Empresas de produção', 'Países de produção',
       'Data de lançamento', 'Receita', 'Duração', 'Idiomas falados', 'Título',
       'Media_de_votos', 'Contagem_de_votos'],
      dtype='object')

# <font color="#10c586" style="font-size: 30px;">2. Codificação one-hot para variáveis categóricas</font>
<hr style="border: 2px solid #10c586;">


## <font style="font-size: 25px;">A codificação one-hot é uma técnica usada para converter variáveis categóricas em uma representação binária, onde cada categoria é representada como uma coluna binária separada. Essa técnica é amplamente utilizada para lidar com variáveis categóricas em algoritmos de aprendizado de máquina, pois permite que o modelo entenda e utilize essas informações de forma adequada.</font>



In [ ]:
for cols in colunas_transformar

# Realizar a codificação one-hot para a coluna
coluna_onde_hot = pd.getdummies()